In [1]:
import numpy as np
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7852\2936652086.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


conclusion: 

Feature Selection: CCI, ADX, Chaikin A/D
Model Selection: best using SVC with poly kernel
Hyperparameter Tuning: d = 2 c = 0.3

Report

1. SVM -> stock price -> goblok tebak all up or down
2. Data collection -> jelasin features dari alphavantage + feature buatan sendiri (short SMA / long SMA ratio)
3. Model Selection -> analyze correlation  bilkang not linearly separable ...
4. Feature Selection
5. Hyperparameter tuning

stevan: 
- feature buatan sendiri (short SMA / long SMA ratio)
- training data selection
- improve acc to >70%

In [2]:
'''
CONST
'''
FILEPATH_STOCK_PRICE = "./tr_eikon_eod_data.csv"
STOCK_NAMES = ['AAPL.O', 'MSFT.O', 'GS.N', 'AMZN.O', 'GS.N']

In [15]:
''' 
UTILS
'''
def sequential_train_test_split(data, ratio = 0.8):
    df = pd.DataFrame(data.dropna()).to_numpy()
    split_index = int(len(df) * ratio)
    return df[:split_index], df

def calculate_return(prediction, data) :
    data = data
    prediction = pd.DataFrame(prediction)
    temp = pd.DataFrame(data)
    temp['log_return'] = np.log(temp / temp.shift(1))
    temp['returne'] = prediction.shift(1) * temp['log_return'].values.reshape(-1, 1)
    temp.dropna(inplace = True) 

    return np.exp(temp[['log_return', 'returne']].sum())


In [4]:
'''
BENCHMARK
'''

def benchmark_accuracy(prediction, test):
    prediction = pd.DataFrame(prediction)
    test = pd.DataFrame(test)

    return (prediction == test).sum() / len(prediction)


In [5]:
''' 
MODEL
'''

def predict_position(train, test):
    # Train Hidden Markov Model
    model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
    model.fit(train.reshape(-1, 1))

    # Predict hidden states for test data
    hidden_states = model.predict(test.reshape(-1, 1))

    # Define a function to map hidden states to rise or fall
    def map_to_rise_or_fall(state):
        return 1 if state == 0 else -1

    # Map hidden states to rise or fall
    predicted_rise_fall = np.array(list(map(map_to_rise_or_fall, hidden_states)))

    return predicted_rise_fall

In [16]:
apple_data = pd.read_csv('intel_output.csv', index_col = 0, parse_dates = True).dropna()
apple_data['SMA_EMA_ratio'] = apple_data['SMA'] / apple_data['EMA']
SMA_short = 7
SMA_long = 20
apple_data['SMA_Short'] = apple_data['INTC.O'].rolling(SMA_short).mean()
apple_data['SMA_Long'] = apple_data['INTC.O'].rolling(SMA_long).mean()
apple_data['SMA_Long_Short'] = apple_data['SMA_Short'] / apple_data['SMA_Long']
apple_data['data'] = apple_data['INTC.O']
apple_data['direction'] = np.where(apple_data['INTC.O'] <= apple_data['INTC.O'].shift(-1), 1, -1)

apple_data

,INTC.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data,direction
Date,,,,,,,,,,,,,,,,,,
2010-04-01,20.880,43.4609,40.6623,65.2979,34.4441,36.5832,7.1429,50.0000,4.301096e+08,-5.566023e+09,14.7663,14.6915,1.005091,NaN,NaN,NaN,20.880,-1
2010-06-01,20.800,80.4143,72.8022,46.2398,36.6646,38.3548,71.4286,14.2857,1.571609e+08,-5.086505e+09,14.0551,14.1730,0.991681,NaN,NaN,NaN,20.800,-1
2010-07-01,20.600,9.8465,14.0854,26.2921,28.8873,-127.6594,100.0000,28.5714,-2.592306e+08,-5.198458e+09,13.6668,13.4909,1.013038,NaN,NaN,NaN,20.600,1
2010-11-01,20.950,73.4720,81.3621,66.6020,45.5102,137.0434,35.7143,100.0000,4.548065e+08,-5.178943e+09,13.3050,13.3863,0.993927,NaN,NaN,NaN,20.950,-1
2010-12-01,20.608,63.8344,65.5033,65.2722,30.9812,151.4882,7.1429,14.2857,4.755292e+08,-5.071694e+09,14.2613,14.2928,0.997796,NaN,NaN,NaN,20.608,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,50.710,16.5231,21.8288,32.3000,22.1620,-153.6915,100.0000,0.0000,2.904198e+09,-1.487095e+09,46.2450,45.9993,1.005341,52.874286,54.5605,0.969095,50.710,-1
2018-06-26,49.670,15.5695,18.6922,28.1699,24.3297,-160.8763,100.0000,0.0000,2.866655e+09,-1.502469e+09,45.8223,45.4383,1.008451,52.097143,54.3065,0.959317,49.670,-1
2018-06-27,48.760,7.8284,13.3070,26.3015,26.7441,-143.1246,100.0000,0.0000,2.833317e+09,-1.535299e+09,45.3378,44.8841,1.010108,51.460000,53.9725,0.953449,48.760,1


In [17]:
scaler = MinMaxScaler()

# Normalize the selected columns
normalized_columns = scaler.fit_transform(apple_data.iloc[:, :-2])

# Create a new DataFrame with the normalized values
normalized_df = pd.DataFrame(normalized_columns, columns=apple_data.columns[:-2])

# Display the normalized DataFrame
apple_data.iloc[:, :-2] = normalized_df
apple_data


,INTC.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data,direction
Date,,,,,,,,,,,,,,,,,,
2010-04-01,0.081672,0.434205,0.397310,0.612997,0.410143,0.522406,0.071429,0.500000,0.205472,0.073369,0.072788,0.069194,0.708089,NaN,NaN,NaN,20.880,-1
2010-06-01,0.079639,0.807347,0.735752,0.362439,0.445922,0.525301,0.714286,0.142857,0.124114,0.178735,0.052965,0.054706,0.352377,NaN,NaN,NaN,20.800,-1
2010-07-01,0.074559,0.094780,0.117448,0.100186,0.320607,0.253994,1.000000,0.285714,0.000000,0.154135,0.042143,0.035646,0.918891,NaN,NaN,NaN,20.600,1
2010-11-01,0.083450,0.737246,0.825889,0.630142,0.588450,0.686582,0.357143,1.000000,0.212834,0.158423,0.032059,0.032723,0.411935,NaN,NaN,NaN,20.950,-1
2010-12-01,0.074762,0.639929,0.658892,0.612659,0.354346,0.710188,0.071429,0.142857,0.219010,0.181989,0.058713,0.058053,0.514576,NaN,NaN,NaN,20.608,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.839451,0.162198,0.198988,0.179172,0.212243,0.211452,1.000000,0.000000,0.942926,0.969645,0.950167,0.944012,0.714721,0.919781,0.983762,0.269089,50.710,-1
2018-06-26,0.813032,0.152569,0.165959,0.124873,0.247171,0.199710,1.000000,0.000000,0.931735,0.966266,0.938386,0.928336,0.797206,0.899255,0.976842,0.208540,49.670,-1
2018-06-27,0.789915,0.074402,0.109251,0.100309,0.286074,0.228720,1.000000,0.000000,0.921798,0.959053,0.924882,0.912850,0.841166,0.882426,0.967743,0.172200,48.760,1


In [18]:
# apple_data = apple_data[['SlowK', 'RSI', 'CCI', 'Aroon Down', 'OBV', 'direction']]

In [19]:
train_data, test_data = sequential_train_test_split(apple_data)
print(test_data)
X_train, y_train, X_test, y_test = train_data[:, :-1], train_data[:, -1], test_data[:, :-1], test_data[:, -1]

[[ 4.28045218e-02  2.59282484e-01  1.99426311e-01 ...  2.62571115e-01
   1.93500000e+01  1.00000000e+00]
 [ 5.04255049e-02  4.45755115e-01  2.99654397e-01 ...  2.65468612e-01
   1.96500000e+01  1.00000000e+00]
 [ 6.08408485e-02  6.27474799e-01  7.15236446e-01 ...  2.68549785e-01
   2.00600000e+01  1.00000000e+00]
 ...
 [ 7.89914899e-01  7.44022465e-02  1.09251467e-01 ...  1.72199923e-01
   4.87600000e+01  1.00000000e+00]
 [ 8.02362505e-01  7.11164830e-02  7.75901285e-02 ...  1.44929701e-01
   4.92500000e+01  1.00000000e+00]
 [ 8.14048012e-01  1.76247787e-01  8.58216342e-02 ...  1.15640999e-01
   4.97100000e+01 -1.00000000e+00]]


In [10]:
X_train

array([[ 0.04280452,  0.25928248,  0.19942631, ...,  0.0541483 ,
         0.26257111, 19.35      ],
       [ 0.0504255 ,  0.44575511,  0.2996544 , ...,  0.05247279,
         0.26546861, 19.65      ],
       [ 0.06084085,  0.6274748 ,  0.71523645, ...,  0.05146475,
         0.26854978, 20.06      ],
       ...,
       [ 0.45332148,  0.48326169,  0.51311228, ...,  0.51461812,
         0.43385531, 35.51      ],
       [ 0.45128922,  0.339506  ,  0.4773136 , ...,  0.5122615 ,
         0.38666864, 35.43      ],
       [ 0.4441763 ,  0.10174658,  0.29534647, ...,  0.50955071,
         0.35257736, 35.15      ]])

In [20]:
from sklearn.svm import SVC
model = SVC(kernel='rbf', gamma=0.7, C=1.0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
benchmark_accuracy(pred, y_test)

0    0.596895
dtype: float64

In [24]:
best_acc = 0
best_g = -1
best_c = -1
best_return = 0
bestgg = -1
bestcc = -1
log_return = 0  
trades = 0

selected_features = [0, 2, 6, 8, 9, 12, 15]
dat = X_test[:,-1]

for g in range(1, 11, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', gamma=g / 10, C=c / 10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp = calculate_return(pred, dat)
        log_return = temp.values[0]
        if acc[0] > best_acc:
            best_acc = acc[0]
            best_g = g
            best_c = c
        if temp.values[1] > best_return :
            best_return = temp.values[1]
            bestgg = g
            bestcc = c
            trades = (pd.DataFrame(pred).diff().dropna() != 0).sum()

print(f"best acc: {best_acc}, g: {best_g}, c: {best_c}")
print(f"best ret: {best_return}, g: {bestgg}, c: {bestcc}")
print(trades)

best acc: 0.5481798715203426, g: 6, c: 34
best ret: 6.917385773569434, g: 4, c: 36
0    312
dtype: int64


In [22]:
print(log_return)

2.568992248062016


In [23]:
best_acc = 0
best_c = -1
best_return = 0
bestcc = -1

selected_features = [4, 5, 9]
dat = X_test[:,-1]

for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='sigmoid', C= c/10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp = calculate_return(pred, dat)

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    if temp.values[1] > best_return :
        best_return = temp.values[1]
        bestcc = c
    
print(f"best acc: {best_acc}, c: {best_c}")
print(f"best ret: {best_return}, c: {bestcc}")

best acc: 0.5299785867237687, c: 1
best ret: 1436159111.048939, c: 1


In [23]:
best_acc = 0
best_d = -1
best_c = -1
best_return = 0
bestdd = -1
bestcc = -1

selected_features = [1, 5, 11, 13]
dat = X_test[:,-1]

for d in range(1, 5, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree=d, C=c/10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp = calculate_return(pred, dat)

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_d = d
            best_c = c
        if temp.values[1] > best_return :
            best_return = temp.values[1]
            bestdd = d
            bestcc = c
    
print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")
print(f"best ret: {best_return}, d : {bestdd}, c: {bestcc}")

best acc: 0.5508021390374331, d: 1, c: 1
best ret: 1.4417646101490813, d : 2, c: 1


In [24]:
best_acc = 0
best_c = -1
best_return = 0
bestcc = -1

selected_features = [4, 5, 9]

# for d in range(1, 5, 1):
for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='poly', degree=2, C = c / 10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp = calculate_return(pred, dat)

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    if temp.values[1] > best_return :
        best_return = temp.values[1]
        bestcc = c
    
print(f"best acc: {best_acc}, c: {best_c}")
print(f"best ret: {best_return}, c: {bestcc}")

best acc: 0.5508021390374331, c: 1
best ret: 1.4098128190584234, c: 1


In [76]:
# Feature Engineering

# Feature Selection

In [25]:
X = apple_data.dropna().iloc[:, :-1]
y = apple_data.dropna().iloc[:, -1]

In [26]:
X

,INTC.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data
Date,,,,,,,,,,,,,,,,,
2010-08-02,0.042805,0.259282,0.199426,0.353435,0.341465,0.327479,0.928571,0.000000,0.113504,0.156487,0.057375,0.052915,0.769316,0.045826,0.054148,0.262571,19.35
2010-09-02,0.050426,0.445755,0.299654,0.226380,0.621947,0.422984,0.857143,0.142857,0.040647,0.102847,0.004387,0.003577,0.506416,0.044505,0.052473,0.265469,19.65
2010-11-02,0.060841,0.627475,0.715236,0.690430,0.611190,0.624224,0.285714,1.000000,0.194663,0.148336,0.035102,0.034962,0.470214,0.043826,0.051465,0.268550,20.06
2010-12-02,0.070240,0.712561,0.657649,0.623898,0.374316,0.723175,0.000000,0.071429,0.233346,0.187456,0.059616,0.059174,0.500489,0.045750,0.051233,0.293249,20.43
2010-02-16,0.077607,0.874840,0.849056,0.585815,0.316258,0.774199,0.500000,1.000000,0.145721,0.020147,0.023510,0.025403,0.318375,0.050731,0.050920,0.354125,20.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.839451,0.162198,0.198988,0.179172,0.212243,0.211452,1.000000,0.000000,0.942926,0.969645,0.950167,0.944012,0.714721,0.919781,0.983762,0.269089,50.71
2018-06-26,0.813032,0.152569,0.165959,0.124873,0.247171,0.199710,1.000000,0.000000,0.931735,0.966266,0.938386,0.928336,0.797206,0.899255,0.976842,0.208540,49.67
2018-06-27,0.789915,0.074402,0.109251,0.100309,0.286074,0.228720,1.000000,0.000000,0.921798,0.959053,0.924882,0.912850,0.841166,0.882426,0.967743,0.172200,48.76


In [27]:
y

Date
2010-08-02    1
2010-09-02    1
2010-11-02    1
2010-12-02    1
2010-02-16   -1
             ..
2018-06-25   -1
2018-06-26   -1
2018-06-27    1
2018-06-28    1
2018-06-29   -1
Name: direction, Length: 1868, dtype: int32

In [28]:
best_acc = 0
best_ret = 0

dat = X_test[:,-1]
for i in range(len(apple_data.columns) - 1):
    print(X_train[:,i])
    curr_X_train = X_train[:,i].reshape(-1, 1)
    curr_X_test = X_test[:,i].reshape(-1, 1)
    model = SVC(kernel='rbf', gamma= 1.0, C= 1.0)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp =calculate_return(pred, dat).values[1]
    if temp > best_ret:
        best_ret = temp
    
    print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")
print(best_ret)

[0.04280452 0.0504255  0.06084085 ... 0.45332148 0.45128922 0.4441763 ]
feature 0: acc: 0.551 up: 1.0
[0.25928248 0.44575511 0.6274748  ... 0.48326169 0.339506   0.10174658]
feature 1: acc: 0.551 up: 1.0
[0.19942631 0.2996544  0.71523645 ... 0.51311228 0.4773136  0.29534647]
feature 2: acc: 0.551 up: 1.0
[0.35343473 0.22638034 0.69043039 ... 0.22800137 0.16452611 0.14681703]
feature 3: acc: 0.551 up: 1.0
[0.34146499 0.62194741 0.61119042 ... 0.19907673 0.23300248 0.26961748]
feature 4: acc: 0.551 up: 1.0
[0.32747862 0.42298404 0.62422357 ... 0.08882759 0.19668115 0.24859553]
feature 5: acc: 0.551 up: 1.0
[0.928571 0.857143 0.285714 ... 1.       1.       1.      ]
feature 6: acc: 0.529 up: 0.561
[0.       0.142857 1.       ... 0.5      0.428571 0.357143]
feature 7: acc: 0.551 up: 1.0
[0.11350426 0.0406465  0.19466343 ... 0.7731024  0.76545108 0.75924726]
feature 8: acc: 0.551 up: 1.0
[0.15648721 0.10284738 0.14833586 ... 0.78675895 0.78764955 0.78834718]
feature 9: acc: 0.551 up: 1.0
[0

In [31]:
best_acc = 0
best_selected_features = []
best_ret = 0
best_selected_featuresa = []

for i in range(len(apple_data.columns) - 1):
    for j in range(len(apple_data.columns) - 1):
        selected_features = [i, j]
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp =calculate_return(pred, dat).values[1]
        if temp > best_ret:
            best_ret = temp
            best_selected_featuresa = selected_features

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

best acc: 0.5668449197860963 features: [0, 10]
best ret: 1.9564063045541997 features: [0, 11]


In [129]:
print(apple_data.columns)

Index(['AAPL.O', 'SlowK', 'SlowD', 'RSI', 'ADX', 'CCI', 'Aroon Down',
       'Aroon Up', 'OBV', 'Chaikin A/D', 'SMA', 'EMA', 'SMA_EMA_ratio',
       'SMA_Short', 'SMA_Long', 'SMA_Long_Short', 'data', 'direction'],
      dtype='object')


In [14]:
import itertools

all_features = list(range(len(apple_data.columns) - 2))

best_acc = 0
best_selected_features = []
best_ret = 0
best_selected_featuresa = []

for i in range(7, len(apple_data.columns) - 2):
    print(i)
    possible_selected_features = list(itertools.combinations(all_features, i))
    for selected_features in possible_selected_features:
        print(selected_features)
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp =calculate_return(pred, dat).values[1]
        if temp > best_ret:
            best_ret = temp
            best_selected_featuresa = selected_features

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

7
(0, 1, 2, 3, 4, 5, 6)
(0, 1, 2, 3, 4, 5, 7)
(0, 1, 2, 3, 4, 5, 8)
(0, 1, 2, 3, 4, 5, 9)
(0, 1, 2, 3, 4, 5, 10)
(0, 1, 2, 3, 4, 5, 11)
(0, 1, 2, 3, 4, 5, 12)
(0, 1, 2, 3, 4, 5, 13)
(0, 1, 2, 3, 4, 5, 14)
(0, 1, 2, 3, 4, 5, 15)
(0, 1, 2, 3, 4, 6, 7)
(0, 1, 2, 3, 4, 6, 8)
(0, 1, 2, 3, 4, 6, 9)
(0, 1, 2, 3, 4, 6, 10)
(0, 1, 2, 3, 4, 6, 11)
(0, 1, 2, 3, 4, 6, 12)
(0, 1, 2, 3, 4, 6, 13)
(0, 1, 2, 3, 4, 6, 14)
(0, 1, 2, 3, 4, 6, 15)
(0, 1, 2, 3, 4, 7, 8)
(0, 1, 2, 3, 4, 7, 9)
(0, 1, 2, 3, 4, 7, 10)
(0, 1, 2, 3, 4, 7, 11)
(0, 1, 2, 3, 4, 7, 12)
(0, 1, 2, 3, 4, 7, 13)
(0, 1, 2, 3, 4, 7, 14)
(0, 1, 2, 3, 4, 7, 15)
(0, 1, 2, 3, 4, 8, 9)
(0, 1, 2, 3, 4, 8, 10)
(0, 1, 2, 3, 4, 8, 11)
(0, 1, 2, 3, 4, 8, 12)
(0, 1, 2, 3, 4, 8, 13)
(0, 1, 2, 3, 4, 8, 14)
(0, 1, 2, 3, 4, 8, 15)
(0, 1, 2, 3, 4, 9, 10)
(0, 1, 2, 3, 4, 9, 11)
(0, 1, 2, 3, 4, 9, 12)
(0, 1, 2, 3, 4, 9, 13)
(0, 1, 2, 3, 4, 9, 14)
(0, 1, 2, 3, 4, 9, 15)
(0, 1, 2, 3, 4, 10, 11)
(0, 1, 2, 3, 4, 10, 12)
(0, 1, 2, 3, 4, 10, 13)
(0, 1, 2, 3, 4, 

In [34]:

print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")
1-6

best acc: 0.5668449197860963 features: (0, 10)
best ret: 2.3992565838150837 features: (0, 2, 6, 8, 9, 10, 12)
